In [0]:
import tensorflow as tf
import os
import numpy as np
import json
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.collections import PatchCollection
from skimage import io
from skimage.segmentation import quickshift
import glob
import seaborn as sns

In [0]:
import os
import glob
import zipfile
import functools
from skimage import io
from skimage.segmentation import quickshift
import glob
from skimage.transform import rescale, resize, downscale_local_mean

from sklearn.model_selection import train_test_split
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
mpl.rcParams['figure.figsize'] = (12,12)

from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import pandas as pd
from PIL import Image


In [0]:
%matplotlib inline

In [0]:
PATH="drive/inter_iit/conv_data"
img_dir = os.path.join(PATH, "sat")
label_dir = os.path.join(PATH, "gt")

In [0]:
def scale_percentile(matrix):
    """Fixes the pixel value range to 2%-98% original distribution of values"""
    orig_shape = matrix.shape
    matrix = np.reshape(matrix, [matrix.shape[0]*matrix.shape[1], 3]).astype(float)
    
    # Get 2nd and 98th percentile
    mins = np.percentile(matrix, 1, axis=0)
    maxs = np.percentile(matrix, 99, axis=0) - mins
    
    matrix = (matrix - mins[None,:])/maxs[None,:]
    matrix = np.reshape(matrix, orig_shape)
    matrix = matrix.clip(0,1)
    return matrix

def scale_percentile_without_normalization(matrix):
    """Fixes the pixel value range to 2%-98% original distribution of values"""
    orig_shape = matrix.shape
    matrix = np.reshape(matrix, [matrix.shape[0]*matrix.shape[1], 3]).astype(float)
    
    # Get 2nd and 98th percentile
    mins = np.percentile(matrix, 1, axis=0)
    maxs = np.percentile(matrix, 99, axis=0) - mins
    
    matrix = (matrix - mins[None,:])/maxs[None,:]
    matrix = np.reshape(matrix, orig_shape)
    #matrix = matrix.clip(0,1)
    return matrix
def stretch_8bit(bands, lower_percent=2, higher_percent=98):
    out = np.z|eros_like(bands).astype(np.float32)
    for i in range(3):
        a = 0
        b = 1
        c = np.percentile(bands[:, :, i], lower_percent)
        d = np.percentile(bands[:, :, i], higher_percent)
        t = a + (bands[:, :, i] - c) * (b - a) / (d - c)
        t[t < a] = a
        t[t > b] = b
        out[:, :, i] = t
    return out.astype(np.float32)

In [0]:
x_train_filenames = []
y_train_filenames = []
for index in range( 14):
  
  x_train_filenames.append(os.path.join(img_dir, "{}.png".format(index)))
  y_train_filenames.append(os.path.join(label_dir, "{}.png".format(index)))
  #print(index)

print(len(x_train_filenames))  

In [0]:

display_num = 5
nu=len(x_train_filenames)
r_choices = np.random.choice(nu, display_num)

plt.figure(figsize=(10, 15))
for i in range(0, display_num * 2, 2):
  img_num = r_choices[i // 2]
  x_pathname = x_train_filenames[img_num]
  y_pathname = y_train_filenames[img_num]
  
  plt.subplot(display_num, 2, i + 1)
  print(io.imread(x_train_filenames[img_num])[...,0:3].shape)
  plt.imshow((io.imread(x_pathname)[...,0:3]))
  plt.title("Original Image")
  
  example_labels = Image.open(y_pathname)
  label_vals = np.unique(example_labels)
  #print(np.unique(io.imread(y_train_filenames[img_num])))
  plt.subplot(display_num, 2, i + 2)
  plt.imshow(example_labels)
  plt.title("Masked Image")  
  
plt.suptitle("Examples of Images and their Masks")
plt.show()

In [0]:
PATH="drive/inter_iit/split_data"
img_dir = os.path.join(PATH, "sat")
label_dir = os.path.join(PATH, "gt")

In [0]:

h,w=256,256

split_sat=[]
split_gt=[]
t=0
for i in range(0, 14):
  
  img_num=i
  x,y=0,0
  x_pathname = x_train_filenames[img_num]
  y_pathname = y_train_filenames[img_num]
  im_sat=io.imread(x_pathname)[...,0:3]
  im_gt=io.imread(y_pathname)[...,0:3]
  n=i
  print(i)
  
  while 1:
    x=0
    while 1:
      
      fname_lab=os.path.join(label_dir, "{}_{}_{}.png".format(n,x,y))
      fname_img=os.path.join(img_dir, "{}_{}_{}.png".format(n,x,y))
      a,b,c=im_sat[y:y+h, x:x+w].shape
      c,d,f=im_gt[y:y+h, x:x+w].shape
      
      
      if(a==256 and b==256 and c==256 and d==256):
        
        t=t+1
        
        #print(t)
        if(t==1):
          crop_img_x = im_sat[y:y+h, x:x+w]#.flatten()
          a=crop_img_x.reshape(1,h*w*3)
          #df_sat=pd.DataFrame(a)
          crop_img_y = im_gt[y:y+h, x:x+w] 
          b=crop_img_y.reshape(1,h*w*3)
          #df_gt=pd.DataFrame(b)
          #print("dddddddddd")
        else:
          crop_img_x = im_sat[y:y+h, x:x+w]#.flatten()
          a=crop_img_x.reshape(1,h*w*3)
          #temp=pd.DataFrame(a)
          #df_sat = pd.concat([df_sat, temp], axis=0, ignore_index=True)
          #plt.imsave(fname_img,crop_img_x)
          #print(crop_img_y.shape)
          split_sat.append(crop_img_x)

          crop_img_y = im_gt[y:y+h, x:x+w]#.flatten()
          b=crop_img_y.reshape(1,h*w*3)
          #temp2=pd.DataFrame(b)
          #df_gt = pd.concat([df_gt, temp2], axis=0, ignore_index=True)
          #plt.imsave(fname_lab,crop_img_y)
          split_gt.append(crop_img_y)
      x=x+40
     # print(x)
      if(x>=750):
        break
    y=y+40
    if(y>=750):
      break
      
    
  
 

In [0]:

split_sat = np.asarray(split_sat)
split_gt = np.asarray(split_gt)

In [0]:

img_shape = (256, 256, 3)
batch_size = 64
epochs = 5

In [0]:
x_train, x_val, y_train, y_val= train_test_split(split_sat, split_gt, test_size=0.3, random_state=42)

In [0]:
num_train_examples = len(x_train)
num_val_examples = len(x_val)
print(num_train_examples)

In [0]:

from __future__ import division

import numpy as np
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Cropping2D

from keras import backend as K

import keras
import h5py

from keras.layers.normalization import BatchNormalization


from keras.optimizers import Nadam
from keras.callbacks import History
import pandas as pd
from keras.backend import categorical_crossentropy

import datetime
import os

import random
import threading
from tensorflow.python.keras import layers
from keras.models import model_from_json
import tensorflow.contrib as tfcontrib
from tensorflow.python.keras import layers
from tensorflow.python.keras import losses
from tensorflow.python.keras import models
from tensorflow.python.keras import backend as K
from keras import utils

In [0]:
def conv_block(input_tensor, num_filters):
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
  encoder = layers.BatchNormalization()(encoder)
  encoder = layers.Activation('relu')(encoder)
  encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
  encoder = layers.BatchNormalization()(encoder)
  encoder = layers.Activation('relu')(encoder)
  return encoder

def encoder_block(input_tensor, num_filters):
  encoder = conv_block(input_tensor, num_filters)
  encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
  
  return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
  decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
  decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
  decoder = layers.BatchNormalization()(decoder)
  decoder = layers.Activation('relu')(decoder)
  return decoder

In [0]:
inputs = layers.Input(shape=img_shape)

# 256
print(inputs.shape)
encoder0_pool, encoder0 = encoder_block(inputs, 32)
# 128
print(encoder0_pool.shape)
encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64)
# 64

encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128)
# 32

encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256)
# 16

encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)
# 8

center = conv_block(encoder4_pool, 1024)
# center

decoder4 = decoder_block(center, encoder4, 512)
# 16

decoder3 = decoder_block(decoder4, encoder3, 256)
# 32

decoder2 = decoder_block(decoder3, encoder2, 128)
# 64

decoder1 = decoder_block(decoder2, encoder1, 64)
# 128

decoder0 = decoder_block(decoder1, encoder0, 32)

#encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512)
outputs = layers.Conv2D(3, (1, 1), activation='sigmoid')(decoder0)






In [0]:
model = models.Model(inputs=[inputs], outputs=[outputs])

In [0]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

In [0]:
def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

In [0]:
def bce_dice_loss(y_true, y_pred):
    loss = losses.categorical_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss


In [0]:
def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + categorical_crossentropy(y_pred, y_true)

In [0]:
model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_loss])

#model.summary()

In [0]:
save_model_path = 'drive/inter_iit/model/weights.hdf5'
cp = tf.keras.callbacks.ModelCheckpoint(filepath=save_model_path, monitor='val_dice_loss', save_best_only=True, verbose=1)

In [0]:
print(x_val.shape)

In [0]:
im_rows = 256 
im_cols = 256
x_test=x_val
y_test=y_val

im_shape = (im_rows, im_cols, 3)

x_train = x_train.reshape(x_train.shape[0], *im_shape)
y_train = y_train.reshape(y_train.shape[0], *im_shape)
x_test = x_test.reshape(x_test.shape[0], *im_shape)
y_test = x_test.reshape(y_test.shape[0], *im_shape)


In [0]:

print(x_train[1].shape)

In [0]:

plt.imshow(x_train[1])

In [0]:
epochs=5
batch_size=8

In [0]:
from keras.preprocessing.image import ImageDataGenerator
data_augmentation = True
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history=model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=int(np.ceil(num_train_examples / float(batch_size))),
                   epochs=epochs,
                   validation_data=(x_test/255,y_test/255),
                   validation_steps=int(np.ceil(num_val_examples / float(batch_size))),
                   callbacks=[cp])

In [0]:
dice = history.history['dice_loss']
val_dice = history.history['val_dice_loss']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, dice, label='Training Dice Loss')
plt.plot(epochs_range, val_dice, label='Validation Dice Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Dice Loss')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()

In [0]:
# Alternatively, load the weights directly: model.load_weights(save_model_path)
model = models.load_model(save_model_path, custom_objects={'bce_dice_loss': bce_dice_loss,
                                                           'dice_loss': dice_loss})

In [0]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


In [0]:

# Let's visualize some of the outputs 
#data_aug_iter = .make_one_shot_iterator()
#next_element = data_aug_iter.get_next()

# Running next element in our graph will produce a batch of images
plt.figure(figsize=(10, 20))
for i in range(5):
  k=i
  #batch_of_imgs, label = tf.keras.backend.get_session().run(next_element)
  #img = batch_of_imgs[0]
  predicted_label = model.predict(x_test)[k]

  plt.subplot(5, 3, 3 * i + 1)
  plt.imshow(x_test[k])
  plt.title("Input image")
  
  plt.subplot(5, 3, 3 * i + 2)
  plt.imshow(y_test[k])
  plt.title("Actual Mask")
  plt.subplot(5, 3, 3 * i + 3)
  plt.imshow(predicted_label[:, :, :])
  plt.title("Predicted Mask")
plt.suptitle("Examples of Input Image, Label, and Prediction")
plt.show()